In [1]:
import os
import json
import requests
import random

In [3]:
import time

In [4]:
import os
from dotenv import load_dotenv

def configure_environment(dotenv_path: str = None) -> bool:
    """
    从 .env 文件加载环境变量。

    :param dotenv_path: .env 文件的路径。如果为 None，则从当前目录或父目录查找。
    :return: 如果成功加载文件则返回 True，否则返回 False。
    """
    # load_dotenv() 会查找 .env 文件并加载其内容到 os.environ
    # 如果找到了文件并加载成功，它会返回 True
    found = load_dotenv(dotenv_path)
    if not found:
        print("警告: 未找到 .env 文件。请确保该文件存在于项目根目录。")
    return found

## 从本地的env文件中读取环境变量

In [8]:
configure_environment()

True

In [10]:
class GeminiTranslator:
    """
    一个用于与 Google Gemini API 进行单次调用翻译的封装类。
    每次调用都是独立的，不维护对话历史。
    支持代理和自定义安全设置。
    """
    def __init__(self, api_key: str, model_name: str = "gemini-1.5-flash-latest", proxy: str = None):
        """
        初始化翻译器。

        :param api_key: 您的 Google Gemini API 密钥。
        :param model_name: 要使用的模型名称。
        :param proxy: (可选) 本地代理地址，例如 "socks5h://127.0.0.1:1080"。
        """
        if not api_key:
            raise ValueError("API key cannot be empty.")
        self.api_key = api_key
        self.model_name = model_name
        self.api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{model_name}:generateContent"
        
        # 使用 Session 对象可以复用连接，提高性能
        self.session = requests.Session()

        if proxy:
            proxies = {"http": proxy, "https": proxy}
            self.session.proxies.update(proxies)
            print(f"--- [系统] 已配置代理: {proxy} ---")

        self.session.headers.update({
            "Content-Type": "application/json",
            "X-goog-api-key": self.api_key
        })
        
        # 安全配置，防止游戏内容被误拦
        self.safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        ]
        
        # 生成配置，使输出更稳定
        self.generation_config = {
            "temperature": 0.3,
            "topP": 1.0,
            "topK": 32,
            "maxOutputTokens": 8192,
        }
        self.translation_history = []
    def _build_prompt_header(self, task_prompt: str, glossary: dict = None) -> str:
            """
            根据任务描述和词汇表构建完整的初始Prompt。
            """
            header = task_prompt
            if glossary:
                header += "\n\n请严格遵守以下词汇对照表进行翻译，不要自行发挥：\n"
                header += "```\n"
                for eng, chn in glossary.items():
                    header += f"{eng} = {chn}\n"
                header += "```"
            return header
    def _build_single_prompt(self, task_prompt: str, glossary: dict, text_to_translate: str) -> str:
        """
        将任务指令、词汇表和待翻译内容合并成一个完整的单次请求Prompt。
        """
        # 1. 任务指令
        full_prompt = task_prompt
        
        # 2. 词汇表
        if glossary:
            full_prompt += "\n\n请严格遵守以下词汇对照表进行翻译，不要自行发挥：\n"
            full_prompt += "```json\n"
            full_prompt += json.dumps(glossary, indent=2, ensure_ascii=False)
            full_prompt += "\n```\n"

        # 3. 待翻译内容
        full_prompt += "\n以下是待翻译的内容：\n"
        full_prompt += "```\n"
        full_prompt += text_to_translate
        full_prompt += "\n```"
        
        return full_prompt
    def translate(self, text_to_translate: str) -> str | None:
        """
        发送需要翻译的文本，并返回翻译结果。支持上下文。

        :param text_to_translate: 需要翻译的英文文本。
        :return: 翻译后的中文文本，或在出错时返回 None。
        """
        if not self.conversation_history:
            print("[错误] 请先调用 start_new_conversation() 方法初始化会话。")
            return None

        # 将新文本加入对话历史
        self.conversation_history.append({
            "role": "user",
            "parts": [{"text": text_to_translate}]
        })
        
        payload = {"contents": self.conversation_history}
        
        try:
            response = self.session.post(self.api_url, json=payload, timeout=180)
            response.raise_for_status()
            
            response_data = response.json()
            
            if 'candidates' in response_data and response_data['candidates']:
                translated_text = response_data['candidates'][0]['content']['parts'][0]['text']
                
                # 将模型的回复加入历史，以备后续校对
                self.conversation_history.append({
                    "role": "model",
                    "parts": [{"text": translated_text}]
                })
                return translated_text
            else:
                print("--- [警告] 模型未能提供有效翻译，请检查内容 ---")
                print("完整回复:", response_data)
                self.conversation_history.pop() # 移除失败的用户输入
                return None
                
        except requests.exceptions.RequestException as e:
            print(f"发生网络错误: {e}")
            self.conversation_history.pop()
            return None

    def get_conversation_history(self):
        """返回当前的对话历史记录，方便调试。"""
        return self.conversation_history
        
    def start_new_conversation(self, task_prompt: str, glossary: dict = None):
        """
        开始一个新的翻译会话，发送包含任务定义的初始Prompt。

        :param task_prompt: 描述翻译任务的初始提示语。
        :param glossary: 一个英译中的词汇对照表字典。
        """
        # 清空历史记录，开始新的会话
        self.conversation_history = []
        
        # 1. 构建完整的初始Prompt
        initial_prompt = self._build_prompt_header(task_prompt, glossary)
        print("--- [系统] 初始化会话，发送初始Prompt ---")
        # print(initial_prompt)
        print("------------------------------------------")

        # 2. 将初始Prompt作为对话的第一条消息
        self.conversation_history.append({
            "role": "user",
            "parts": [{"text": initial_prompt}]
        })
        
        # 3. 发送请求并获取模型的确认回复
        payload = {"contents": self.conversation_history}
        try:
            response = self.session.post(self.api_url, json=payload, timeout=600)
            response.raise_for_status()  # 如果HTTP状态码是4xx或5xx，则抛出异常
            
            response_data = response.json()
            
            # 安全地提取回复内容
            if 'candidates' in response_data and response_data['candidates']:
                model_response_text = response_data['candidates'][0]['content']['parts'][0]['text']
                
                # 4. 将模型的确认回复也加入历史记录
                self.conversation_history.append({
                    "role": "model",
                    "parts": [{"text": model_response_text}]
                })
                print(f"--- [Gemini] 会话已建立，模型确认 ---")
                print(model_response_text)
                print("------------------------------------------\n")
            else:
                # 处理没有有效回复的情况（例如内容被过滤）
                print("--- [警告] 模型未能提供有效回复，请检查Prompt或API设置 ---")
                print("完整回复:", response_data)
                # 从历史记录中移除失败的user prompt，以便重试
                self.conversation_history.pop()

        except requests.exceptions.RequestException as e:
            print(f"发生网络错误: {e}")
            # 从历史记录中移除失败的user prompt
            self.conversation_history.pop()
            
    def translate_single_shot(self, task_prompt: str, text_to_translate: str, glossary: dict = None) -> str | None:
        """
        在一次API调用中完成翻译任务。

        :param task_prompt: 描述翻译任务的初始提示语。
        :param text_to_translate: 需要翻译的英文文本。
        :param glossary: (可选) 一个英译中的词汇对照表字典。
        :return: 翻译后的中文文本，或在出错时返回 None。
        """
        # 构建包含所有信息的单个Prompt
        final_prompt = self._build_single_prompt(task_prompt, glossary, text_to_translate)
        
        # 构建API请求的payload
        payload = {
            "contents": [{"parts": [{"text": final_prompt}]}],
            "safetySettings": self.safety_settings,
            "generationConfig": self.generation_config,
        }

        print("--- [系统] 正在发送单次翻译请求... ---")
        # print("--- [调试] 发送的完整Prompt如下: ---\n", final_prompt) # 如果需要调试，可以取消此行注释
        
        try:
            response = self.session.post(self.api_url, json=payload, timeout=3000)

            if not response.ok:
                print(f"--- [错误] 服务器返回状态码: {response.status_code} ---")
                try:
                    error_data = response.json()
                    print("服务器错误详情:", json.dumps(error_data, indent=2, ensure_ascii=False))
                except json.JSONDecodeError:
                    print("服务器原始响应:", response.text)
                return None

            response.raise_for_status()
            
            response_data = response.json()
            
            # 检查响应的有效性
            if 'candidates' in response_data and response_data['candidates']:
                candidate = response_data['candidates'][0]
                if 'finishReason' in candidate and candidate['finishReason'] in ['SAFETY', 'RECITATION']:
                    print(f"--- [错误] 模型生成的内容被阻止，原因: {candidate['finishReason']} ---")
                    print("安全评级详情:", candidate.get('safetyRatings'))
                    return None
                
                translated_text = candidate['content']['parts'][0]['text']
                self.translation_history.append(translated_text)
                return translated_text
            else:
                # 检查输入是否被阻止
                if 'promptFeedback' in response_data and 'blockReason' in response_data:
                    reason = response_data['promptFeedback']['blockReason']
                    print(f"--- [错误] 您的输入内容被安全策略阻止，原因: {reason} ---")
                else:
                    print("--- [警告] 模型未能提供有效翻译，响应中不包含'candidates'字段 ---")
                    print("完整回复:", response_data)
                return None
                
        except requests.exceptions.HTTPError as http_err:
            print(f"发生HTTP错误: {http_err}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"发生网络错误: {e}")
            return None

## 初始化翻译器

In [13]:
# --- 1. 初始化 ---
# 强烈建议从环境变量获取API密钥
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    print("请设置 GEMINI_API_KEY 环境变量！")
else:
    api_key = os.getenv("GEMINI_API_KEY")
    proxy_address = os.getenv("PROXY_URL") # 新增：获取代理地址
    
    translator = translator = GeminiTranslator(api_key=api_key,
                                               # proxy=proxy_address
                                              )
    print(f'翻译器初始化完成.')

翻译器初始化完成.


## prompt和词汇表配置

In [16]:
# # 读取mod中的翻译对照词汇
# en_gamestring = [i.strip() for i in open('../data/gamestring/GameString-enUS.txt').readlines()]
# cn_gamestring = [i.strip() for i in open('../data/gamestring/GameString-zhCN.txt',encoding='utf-8').readlines()]
# en_gs_map = {j[0]:j[1] for j in [i.split('=') for i in en_gamestring]}
# cn_gs_map = {j[0]:j[1] for j in [i.split('=') for i in cn_gamestring]}

In [18]:
# en_cn_vocab = {en_gs_map[k]:cn_gs_map[k] for k in cn_gs_map}

In [20]:
# --- 2. 定义翻译任务和词汇表 ---
# 定义初始Prompt，告诉AI它的角色和任务规则
my_task_prompt = """
我正在进行游戏内的文本汉化工作,其中数据格式为KEY=VALUE其中key为游戏内索引对应文本所用的键,而value则为相应的文本.
你现在需要对这样格式的文本进行翻译,有以下几个要求:
key必须保持原样,不能做任何修改.
value中可能存在控制类的字符,例如</s>,<color>等, 对于这些字符保持原样不要做翻译
翻译的结果也需要为KEY=VALUE的形式,其中value为翻译之后的中文内容(包含控制字符不变)
翻译之后的条目必须与原始输入的条目在顺序和数目上一致.
但是当翻译内容较长时，难免会出现一些控制字符被截断，以至于后面的内容没有被正确的翻译出来。
例如以下的内容：
```
Terrain/Name/Kit@Redstone=Kit@Redstone
UI/ArchivePlayCinematic=<h/>PLAY CINEMATIC
UI/ArchivePlayMission=<h/>REPLAY
UI/Building=Building
UI/HarvesterCountMinerals=<img path="@@UI/ResourceIconNoBackground0" alignment="AbsoluteMiddle" width="20" height="20"/> Workers:<c val="%colorValue%"> %current%/%ideal%</c>
UI/MercenaryPanelNoSelection=Select a Mercenary
UI/Mutating=Improving
翻译之后的文本：
```
```
Terrain/Name/Kit@Redstone=Kit@Redstone
UI/ArchivePlayCinematic=<h/>播放过场动画
UI/ArchivePlayMission=<h/>回放
UI/Building=建筑物
UI/HarvesterCountMinerals=<img path
UI/MercenaryPanelNoSelection=选择一名雇佣兵
UI/Mutating=改进中
```
其中UI/ArchivePlayCinematic部分则缺失了。请注意其中格式问题，在翻译时一定保留这些格式化的字符串。
下面将会给出待翻译的文本，请翻译为中文，注意上述问题。
"""

# 定义你的专属词汇表，确保关键术语翻译统一
# my_glossary = en_cn_vocab

## 测试翻译内容

In [23]:
def preprocess_gamestring_text(fpath:str):
    file_content = open(fpath,'rb').read()
    file_nobom_content = file_content.decode("utf-8-sig").encode("utf-8")
    return file_nobom_content.decode('utf-8')

In [25]:
def postprocess_gamestring_text(gs_text:str):
    return gs_text.replace('```','').strip()

In [27]:
def raw_to_gs_map(raw_str):
    content_lst = [i for i in raw_str.split('\n') if not i == '']
    content_split = [i.split('=') for i in content_lst]
    return {i[0]:'='.join(i[1:]) for i in content_split}

In [92]:
en_raw_dict['Behavior/Tooltip/ConsulDamageBoostStacker']

'Unit is currently stacking shots and will deal 30x more damage in <d ref="Unit,ConsulRange,Behavior,ConsulDamageBoostStacker,StackCount - 15"/> shots.\r'

In [94]:
cn_raw_dict['Behavior/Tooltip/ConsulDamageBoostStacker']

'单位当前正在叠加射击，并将在<d ref="Unit,ConsulRange,Behavior,ConsulDamageBoostStacker,StackCount - 15"/>次射击后造成30倍伤害。\r'

## 变量设置

In [30]:
mod_folder_path = '../data/mods/RebelsofLiberty.SC2Mod/'

In [32]:
batch_size = 500

In [34]:
base_maps_path = '../data/maps/'
base_enus_folder = 'enUS.SC2Data'
base_zhcn_folder = 'zhCN.SC2Data'
local_folder = 'LocalizedData'
gs_fname = 'GameStrings.txt'

## MOD文本翻译

In [37]:
mod_en_raw_content = preprocess_gamestring_text(os.path.join(mod_folder_path,base_enus_folder,local_folder,gs_fname))
mod_en_raw_lst = [i.strip() for i in mod_en_raw_content.split('\r\n') if not i=='']

In [39]:
mod_en_raw_dict = {i.split('=')[0]:i.split('=')[1] for i in mod_en_raw_lst}

### 初始化翻译器第一轮对话

In [104]:
translator.start_new_conversation(task_prompt=my_task_prompt)

--- [系统] 初始化会话，发送初始Prompt ---

我正在进行游戏内的文本汉化工作,其中数据格式为KEY=VALUE其中key为游戏内索引对应文本所用的键,而value则为相应的文本.
你现在需要对这样格式的文本进行翻译,有以下几个要求:
key必须保持原样,不能做任何修改.
value中可能存在控制类的字符,例如</s>,<color>等, 对于这些字符保持原样不要做翻译
翻译的结果也需要为KEY=VALUE的形式,其中value为翻译之后的中文内容(包含控制字符不变)
翻译之后的条目必须与原始输入的条目在顺序和数目上一致.
但是当翻译内容较长时，难免会出现一些控制字符被截断，以至于后面的内容没有被正确的翻译出来。
例如以下的内容：
code
Code
Terrain/Name/Kit@Redstone=Kit@Redstone
UI/ArchivePlayCinematic=<h/>PLAY CINEMATIC
UI/ArchivePlayMission=<h/>REPLAY
UI/Building=Building
UI/HarvesterCountMinerals=<img path="@@UI/ResourceIconNoBackground0" alignment="AbsoluteMiddle" width="20" height="20"/> Workers:<c val="%colorValue%"> %current%/%ideal%</c>
UI/MercenaryPanelNoSelection=Select a Mercenary
UI/Mutating=Improving
UI/Reset_Control=<h/>RESET
UI/TeamColor01=Red
UI/TeamColor07=Aquamarine
UI/TeamColor08=Light Purple
翻译之后的文本：
code
Code
Terrain/Name/Kit@Redstone=Kit@Redstone
UI/ArchivePlayCinematic=<h/>播放过场动画
UI/ArchivePlayMission=<h/>回放
UI/Building=建筑物
UI/HarvesterCountMinerals=<img path
UI/Mercen

In [41]:
mod_gs_keys = list(mod_en_raw_dict.keys())
mod_cn_result_dict = {}
missing_keys_inresponse = []
for i in range(0,len(mod_en_raw_dict),batch_size):
    print(f'Batch:{i} size:{batch_size} apply for translation.')
    
    keys_for_trans = set(mod_gs_keys[i:i+batch_size])
    en_content_for_trans = '\n'.join([f'{key}={mod_en_raw_dict[key]}' for key in keys_for_trans])
    
    cn_result_content = translator.translate_single_shot(task_prompt=my_task_prompt,
                                                         text_to_translate=en_content_for_trans)

    print('Respone content parsing and checking missing keys.')
    cn_result_batch_dict = raw_to_gs_map(postprocess_gamestring_text(cn_result_content))

    count_missing_key = 0
    for postkey in keys_for_trans:
        if postkey not in cn_result_batch_dict:
            count_missing_key += 1
            missing_keys_inresponse.append(postkey)

    print(f'Missing:{count_missing_key} Merge to result cn dict.')
    mod_cn_result_dict = {**mod_cn_result_dict, ** cn_result_batch_dict}
    

Batch:0 size:500 apply for translation.
--- [系统] 正在发送单次翻译请求... ---
Respone content parsing and checking missing keys.
Missing:1 Merge to result cn dict.
Batch:500 size:500 apply for translation.
--- [系统] 正在发送单次翻译请求... ---
Respone content parsing and checking missing keys.
Missing:20 Merge to result cn dict.
Batch:1000 size:500 apply for translation.
--- [系统] 正在发送单次翻译请求... ---
Respone content parsing and checking missing keys.
Missing:192 Merge to result cn dict.
Batch:1500 size:500 apply for translation.
--- [系统] 正在发送单次翻译请求... ---
Respone content parsing and checking missing keys.
Missing:171 Merge to result cn dict.
Batch:2000 size:500 apply for translation.
--- [系统] 正在发送单次翻译请求... ---
Respone content parsing and checking missing keys.
Missing:183 Merge to result cn dict.
Batch:2500 size:500 apply for translation.
--- [系统] 正在发送单次翻译请求... ---
Respone content parsing and checking missing keys.
Missing:204 Merge to result cn dict.
Batch:3000 size:500 apply for translation.
--- [系统] 正在发送单次翻

In [65]:
import re
from collections import OrderedDict

In [106]:
# 辅助函数：解析文本为有序字典 (保持不变)
def parse_localization_text(text: str) -> OrderedDict:
    """
    将 KEY=VALUE 格式的文本解析为有序字典。
    使用 OrderedDict 来保证条目顺序不变。
    """
    data = OrderedDict()
    lines = text.strip().split('\n')
    for line in lines:
        if '=' in line:
            key, value = line.split('=', 1)
            data[key.strip()] = value
    return data

# 辅助函数：提取所有格式化标签 (保持不变)
def extract_tags(value: str) -> list:
    """
    从文本值中提取所有类XML格式的标签。
    例如 <h/>, <color="red">, </color>, <img ... />
    """
    tag_pattern = r"<[^>]+>"
    return re.findall(tag_pattern, value)


def find_mismatched_key(original_text: str, translated_text: str) -> list[str]:
    """
    校对原文和译文，并返回所有格式化标签不匹配的条目的Key列表。

    :param original_text: 英文原文字符串。
    :param translated_text: 中文翻译字符串。
    :return: 一个包含所有格式不匹配的Key的列表 (list[str])。
    :raises ValueError: 如果原文和译文的Key列表在数量或顺序上不匹配，会抛出此异常。
    """
    mismatched_keys = []
    
    # 1. 解析原文和译文
    original_data = parse_localization_text(original_text)
    translated_data = parse_localization_text(translated_text)
    
    # 2. 检查Key的完整性和顺序，这是后续操作的基础
    original_keys = list(original_data.keys())
    translated_keys = list(translated_data.keys())
    
    if original_keys != translated_keys:
        # 构造详细的错误信息
        missing_keys = set(original_keys) - set(translated_keys)
        extra_keys = set(translated_keys) - set(original_keys)
        error_msg = "严重错误：原文和译文的Key列表不匹配，无法进行比较。"
        if missing_keys:
            error_msg += f"\n  - 译文缺失的Key: {list(missing_keys)}"
            print(error_msg)
            return {key:original_data[key] for key in missing_keys}
        if extra_keys:
            error_msg += f"\n  - 译文多出的Key: {list(extra_keys)}"
            print(error_msg)
            return {key:original_data[key] for key in extra_keys}
        # if not missing_keys and not extra_keys:
        #     error_msg += "\n  - Key的数量一致，但顺序不一致。"
        # raise ValueError(error_msg)

    # 3. 逐条对比格式化标签
    for key, original_value in original_data.items():
        translated_value = translated_data[key]
        
        original_tags = extract_tags(original_value)
        translated_tags = extract_tags(translated_value)
        
        # 如果标签列表不匹配
        if original_tags != translated_tags:
            # 将这个Key添加到结果列表中
            mismatched_keys.append(key)
            
    return {key:original_data[key] for key in mismatched_keys}

In [198]:
en_raw_text = preprocess_gamestring_text('../data/mods/RebelsofLiberty.SC2Mod/enUS.SC2Data/LocalizedData/GameStrings.txt')
cn_raw_text = preprocess_gamestring_text('../data/mods/RebelsofLiberty.SC2Mod/zhCN.SC2Data/LocalizedData/GameStrings.txt')

In [200]:
en_raw_dict = raw_to_gs_map(en_raw_text)
cn_raw_dict = raw_to_gs_map(cn_raw_text)

In [202]:
len(cn_raw_dict)

11389

In [186]:
cn_raw_dict = raw_to_gs_map(preprocess_gamestring_text('./saved.txt'))

In [188]:
cn_new_dict = raw_to_gs_map(preprocess_gamestring_text('./missing_text_cn.txt'))

In [190]:
for key in cn_new_dict:
    cn_raw_dict[key] = cn_new_dict[key]

In [204]:
formate_mismatched_dict = find_mismatched_key(original_text=en_raw_text, translated_text=cn_raw_text)

## 处理缺失的key

In [55]:
def trans_missing_text(batch_size, missing_keys, en_gs_dict,):
    '''
    校对翻译的中文内容的key对应英文文本之间格式是否一致
    '''
    checked_cn_dict = {}
    translator.start_new_conversation(task_prompt=my_task_prompt)
    
    while len(missing_keys)!=0:
        print(f'Len:{len(missing_keys)} Checkd:{len(checked_cn_dict)} size:{batch_size} missing keys apply for translation.')
        
        keys_for_trans = set(missing_keys[0:0+batch_size])
        
        en_content_for_trans = '\n'.join([f'{key}={en_gs_dict[key]}' for key in keys_for_trans])
        
        cn_result_content = translator.translate(en_content_for_trans)
        
        print('Respone content parsing and checking missing keys.')
        print(cn_result_content)
        cn_result_batch_dict = raw_to_gs_map(postprocess_gamestring_text(cn_result_content))
        print(cn_result_batch_dict)
        for key, original_value in en_gs_dict.items():
            if not key in cn_result_batch_dict:
                continue
            translated_value = cn_result_batch_dict[key]
            
            original_tags = extract_tags(original_value)
            translated_tags = extract_tags(translated_value)
            
            # 如果标签列表不匹配
            if original_tags != translated_tags:
                # 将这个Key添加到结果列表中
                mismatched_keys.append(key)
            else:
                checked_cn_dict[key] = translated_value
            
            missing_keys = [i for i in missing_keys if not i in checked_cn_dict]
        
    return checked_cn_dict

In [63]:
# trans_missing_text(batch_size=100,missing_keys=list(formate_mismatched_dict.keys()), en_gs_dict=formate_mismatched_dict)

In [178]:
def write_to_zhcn_local(parent_folder:str,# mod或者map文件夹
                       en_gs_dict:dict,
                       cn_gs_dict:dict):
    # 首先校验key是否存在
    for en_key in en_gs_dict:
        if not en_key in cn_gs_dict:
            print(f'Missing key:{en_key}')
    cnzh_path = os.path.join(parent_folder, base_zhcn_folder, local_folder)
    os.makedirs(cnzh_path,exist_ok=True)
    content = [f'{key}={cn_gs_dict[key]}\n' for key in en_gs_dict]
    with open(os.path.join(cnzh_path, gs_fname),'w',encoding='utf-8') as file:
        file.writelines(content)
    return True

In [196]:
write_to_zhcn_local('../data/mods/RebelsofLiberty.SC2Mod/',
                    en_gs_dict=en_raw_dict,
                    cn_gs_dict=cn_raw_dict)

True

In [194]:
cn_raw_dict['UserData/SceneTypes/CutScene_Name'] = '场景剪辑'

In [184]:
len(cn_raw_dict)

1

## 地图文本翻译

In [ ]:
# for map_name in os.listdir(base_maps_path):
#     if not '.SC2Map' in map_name:
#         continue
#     print(f'Translation GameStrings for map:{map_name}.')
#     gs_en_raw_text = preprocess_gamestring_text(os.path.join(base_maps_path, map_name, base_enus_folder, local_folder, gs_fname))

#     translator = translator = GeminiTranslator(api_key=api_key,
#                                                    # proxy=proxy_address
#                                                   )
#     gs_cn_raw_response = translator.translate_single_shot(task_prompt=my_task_prompt,
#                                      text_to_translate=gs_en_raw_text,
#                                      glossary=my_glossary)
    
#     print(f'Translation finished, preparing write out.')
#     gs_cn_raw_text = postprocess_gamestring_text(gs_cn_raw_response)
    
#     # 验证key是否不存在
#     result_cn_gs_map = raw_to_gs_map(response_trans_text)
    
#     origin_en_gs_map = raw_to_gs_map(gs_en_raw_text)

#     for key in origin_en_gs_map:
#         if not key in result_cn_gs_map:
#             print(f'In map {map_name}, key {key} not found in translation content.')

#     # 写出文件
#     cn_fpath = os.path.join(base_maps_path, map_name, base_zhcn_folder, local_folder, gs_fname)
#     cn_gs_content = '\n'.join([f'{key}={result_cn_gs_map[key]}' for key in origin_en_gs_map if key in result_cn_gs_map])

#     with open(cn_fpath,'w',encoding='utf-8') as file:
#         file.write(cn_gs_content)
        
#     waiting_second = random.randint(10,60)
#     print(f'Successfully writing out for {map_name} then random waiting for:{waiting_second}second.')
#     time.sleep(waiting_second)